In [1]:
import pandas as pd
import numpy as np
import sys
import math
import csv
import urllib
import collections

#import scikit-learn libraries for implementation of machine learning algorithms
import sklearn

In [2]:
#recursive feature elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

#feature importance
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier

# to create predictions
from sklearn.cross_validation import train_test_split

#algorithms tested
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import tree
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
epl9318 = pd.read_csv("C:\gusdk\deep\EPL_Set.csv", sep = ",",engine='python')

In [5]:
epl9318.shape

(9674, 76)

In [6]:
epl9318.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Season', 'Winner', 'Loser', 'Referee', 'HS', 'AS', 'HST', 'AST',
       'HHW', 'AHW', 'HC', 'AC', 'HF', 'AF', 'HO', 'AO', 'HY', 'AY', 'HR',
       'AR', 'HBP', 'ABP', 'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33',
       'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36', 'Unnamed: 37',
       'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40', 'Unnamed: 41',
       'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44', 'Unnamed: 45', 'Bb1X2',
       'BbMxH', 'BbAvH', 'BbMxD', 'BbAvD', 'BbMxA', 'BbAvA', 'BbOU',
       'BbMx>2.5', 'BbAv>2.5', 'BbMx<2.5', 'BbAv<2.5', 'BbAH', 'BbAHh',
       'BbMxAHH', 'BbAvAHH', 'BbMxAHA', 'BbAvAHA', 'PSCH', 'PSCD', 'PSCA',
       'Unnamed: 67', 'Unnamed: 68', 'Unnamed: 69', 'Unnamed: 70',
       'Unnamed: 71', 'Unnamed: 72', 'Unnamed: 73', 'Unnamed: 74',
       'Unnamed: 75'],
      dtype='object')

In [7]:
epl9318 = epl9318[["Date","HomeTeam","AwayTeam","FTHG","FTAG","FTR","Season","Winner","Loser"]]
epl9318.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Season,Winner,Loser
0,14/08/1993,Arsenal,Coventry,0,3,A,1993,Coventry,Arsenal
1,14/08/1993,Aston Villa,QPR,4,1,H,1993,Aston Villa,QPR
2,14/08/1993,Chelsea,Blackburn,1,2,A,1993,Blackburn,Chelsea
3,14/08/1993,Liverpool,Sheffield Weds,2,0,H,1993,Liverpool,Sheffield Weds
4,14/08/1993,Man City,Leeds,1,1,D,1993,None,None


In [8]:
a = epl9318[epl9318['Season'] == 1993]
a = a[a['Winner'] == 'Chelsea']
b = len(a)
b

13

In [9]:
teamlist = epl9318['HomeTeam'].unique()
teamlistlen = len(teamlist)
teamlist[2]

'Chelsea'

In [10]:
def getAnnualTeamData(teamname,season):

    annual_data = epl9318[epl9318['Season'] == season]
    
    # num goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamname]
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamname]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    
    #total goals allowed
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #games won percentage = games won / (games won + games lost)
    gamesWon = annual_data[annual_data["Winner"] == teamname] 
    gamesLost = annual_data[annual_data["Loser"] == teamname]
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
        
    if numGames == 0:
        gamesWon = 0
        gamesLost = 0
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        gamesWonPercentage = 0
    
    return [totalGoalsScored, totalGoalsAllowed, gamesWonPercentage]

In [11]:
getAnnualTeamData('Chelsea',2015)

[59, 53, 0.3157894736842105]

In [12]:
#create a dictionary for all the team stats in a year for all the teams
def createAnnualDict(year):
    annualDictionary = collections.defaultdict(list)
    for i in range(teamlistlen):
        team = teamlist[i]
        team_vector = getAnnualTeamData(team, year)
        annualDictionary[team] = team_vector
    
    return annualDictionary 

In [13]:
createAnnualDict(2015)

defaultdict(list,
            {'Arsenal': [65, 36, 0.5263157894736842],
             'Aston Villa': [27, 76, 0.07894736842105263],
             'Chelsea': [59, 53, 0.3157894736842105],
             'Liverpool': [63, 50, 0.42105263157894735],
             'Man City': [71, 41, 0.5],
             'Newcastle': [44, 65, 0.23684210526315788],
             'Oldham': [0, 0, 0],
             'Sheffield United': [0, 0, 0],
             'Southampton': [59, 41, 0.47368421052631576],
             'West Ham': [65, 51, 0.42105263157894735],
             'Norwich': [39, 67, 0.23684210526315788],
             'Tottenham': [69, 35, 0.5],
             'Everton': [59, 55, 0.2894736842105263],
             'Ipswich': [0, 0, 0],
             'Leeds': [0, 0, 0],
             'Wimbledon': [0, 0, 0],
             'Blackburn': [0, 0, 0],
             'Coventry': [0, 0, 0],
             'Man United': [49, 35, 0.5],
             'QPR': [0, 0, 0],
             'Sheffield Weds': [0, 0, 0],
             'Swindon': [

In [14]:
def getTrainingData(years):
    totalNumGames = 0
    for year in years:
        annual = epl9318[epl9318['Season'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData('Arsenal',2015))
    xTrain = np.zeros((totalNumGames, numFeatures))
    yTrain = np.zeros((totalNumGames))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict(year)
        annual = epl9318[epl9318['Season'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros((numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros((numGamesInYear))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a,b, in zip(h_vector,a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                yTrainAnnual[counter] = 1
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [-p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInYear + indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear + indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

In [15]:
years = range(1993,2015)
xTrain, yTrain = getTrainingData(years)
np.save('xTrain',xTrain)
np.save('yTrain',yTrain)

In [16]:
xTrain.shape

(8524, 3)

In [17]:
yTrain.shape

(8524,)

In [18]:
#Tried all the following models
#Im = linear_model.LinearRegression() #0.49
#Im = tree.DecisionTreeClassifier() #0.41
#Im = tree.DecisionTreeRegressor() #0.502
Im = linear_model.LogisticRegression() #0.38
#Im = linear_model.BayesianRidge() #0.498
#m = linear_model.Lasso() #0.495
#Im = svm.SVC() #0.518
#Im = svm.SVR() #0.5
#Im = linear_model.Ridge(alpha = 0.5) #0.505
#Im = AdaBoostClassifier(n_estimators = 100) #0.471
####Im = GradientBoostingClassifer(n_estimators = 100)
#Im = GradientBoostingRegressor(n_estimators = 100, max_depth = 9) #0.496
#m = RandomForestClassifier(n_estimators = 100) #0.505
#Im = KNeighborsClassifier(n_neighbors = 30) #not possible with only 10 features

In [19]:
xTrain, X_test, yTrain, Y_test = train_test_split(xTrain,yTrain)
print (xTrain.shape, yTrain.shape)
print (X_test.shape, Y_test.shape)

model2 = Im.fit(xTrain,yTrain)
predictions = Im.predict(X_test)

print ((sum(predictions))/(len(predictions)))

(6393, 3) (6393,)
(2131, 3) (2131,)
0.5771938057250118


In [20]:
def createGamePrediction(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, Y_test = train_test_split(xTrain, yTrain)
    Im = linear_model.LogisticRegression()
    model2 = Im.fit(xTrain, yTrain)
    diff = [ a - b for a,b in zip(team1_vector, team2_vector)]
    predictions = Im.predict([diff])
    predictions = ((sum(predictions))/(len(predictions)))
    return predictions

In [21]:
team1_vector = getAnnualTeamData('Aresenal',2017)
team2_vector = getAnnualTeamData('Chelsea',2017)
team3_vector = getAnnualTeamData('Man City',2017)

print ('Probability that ' + 'Aresenal' + ' wins : ', createGamePrediction(team1_vector, team2_vector, xTrain, yTrain))
print ('Probability that ' +'Chelsea' + ' wins : ', createGamePrediction(team2_vector, team1_vector, xTrain, yTrain))

Probability that Aresenal wins :  1.0
Probability that Chelsea wins :  1.0


In [22]:
epltest = pd.read_csv("C:\\gusdk\\deep\\EPL_Set.csv", sep = ",",engine='python')

In [23]:
team1_vector = getAnnualTeamData('Arsenal',2017)
team2_vector = getAnnualTeamData('Crystal Palace',2017)
print (createGamePrediction(team1_vector, team2_vector, xTrain, yTrain))

1.0


In [24]:
team1_vector = getAnnualTeamData('Burnley',2017)
team2_vector = getAnnualTeamData('Man United',2017)
print (createGamePrediction(team1_vector, team2_vector, xTrain, yTrain))

1.0


In [25]:
len(epltest.index)

9674

In [33]:
def formulatePrediction():
    probs = [[0 for x in range(3)] for x in range(len(epltest.index))]
    for index, row in epltest.iterrows():
        year = row['Season']
        team1_Name = row['HomeTeam']
        team2_Name = row['AwayTeam']
        team1_vector = getAnnualTeamData(team1_Name,year)
        team2_vector = getAnnualTeamData(team2_Name,year)
        prediction = createGamePrediction(team1_vector, team2_vector, xTrain, yTrain)
        probs[index][0] = team1_Name
        probs[index][1] = team2_Name
        probs[index][2] = prediction
    probs = pd.np.array(probs)
    return probs

In [34]:
a = formulatePrediction()
print (a)

[['Arsenal' 'Coventry' '1.0']
 ['Aston Villa' 'QPR' '1.0']
 ['Chelsea' 'Blackburn' '0.0']
 ...
 ['Arsenal' 'Man City' '0.0']
 ['Liverpool' 'West Ham' '1.0']
 ['Southampton' 'Burnley' '1.0']]
